In [33]:
# Notebook vorbereiten
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
import shutil 
ab_ag_raw = pd.read_csv('data/ab_ag.tsv', sep='\t')
columns = pd.read_csv('data/columns.tsv', sep='\t')
uniprot_data = pd.read_csv('data/uniprot_data.tsv', sep='\t')

# ab_ag aus ab_ag_raw aufbereiten
ab_ag = ab_ag_raw[["pdb", "Hchain", "Lchain", "model", "antigen_chain", "antigen_name", "antigen_species"]]
# Füge Spalten für die CDR-Sequenzen in ab_ag hinzu
for col in ["CDR_H1", "CDR_H2", "CDR_H3", "CDR_L1", "CDR_L2", "CDR_L3"]:
    if col not in ab_ag.columns:
        ab_ag[col] = np.nan
    ab_ag[col] = ab_ag[col].astype(object)
# Entferne Einträge ohne Antigennamen
ab_ag.dropna(subset=["antigen_chain"], inplace=True)

# Ich hab aus Versehen das Code-Feld gelöscht! Code von Anna für doppelte pdb Einträge zurückholen!

C:\Users\marce\AppData\Local\Temp\ipykernel_13884\800876450.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag[col] = np.nan
C:\Users\marce\AppData\Local\Temp\ipykernel_13884\800876450.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ab_ag[col] = ab_ag[col].astype(object)
C:\Users\marce\AppData\Local\Temp\ipykernel_13884\800876450.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cav

In [34]:
# Überarbeiteter Code zum Herunterladen der Antikörpersequenzen. Direkt in ab_ag_raw speichern
# Funktionen aus dem sabdab_downloader importieren
from common.sabdab_downloader import getannotation

# Definiere CDR-Regionen als slices für die H- und L-Ketten (Literatur überprüfen, ob die Indizes korrekt sind!)
cdr_regions = {
        "H1": slice(26, 32),
        "H2": slice(52, 56),
        "H3": slice(95, 102),
        "L1": slice(24, 34),
        "L2": slice(50, 56),
        "L3": slice(89, 97)
}

# Lade die Chothia-Daten und Chothia-nummerierten Sequenzen für jeden PDB-Eintrag herunter und speichere sie im Ordner des entsprechenden PDB-Eintrags
for x in ab_ag.index:
    # Zeilenweise Einträge für den aktuellen PDB-Eintrag extrahieren (für die Chothia-annotierten Sequenzen)
    fab_unique = ab_ag.loc[x,["pdb", "Hchain", "Lchain", "model"]].tolist()
    pdb = fab_unique[0] # PDB-ID aus der Liste extrahieren
    # Erstelle den Ordner für die Chothia-annotierte Sequenz des PDB-Eintrags, falls er nicht existiert
    if not os.path.exists(f"data/sabdab_download/{pdb}"):
        os.makedirs(f"data/sabdab_download/{pdb}")
    # Lade die Chothia-nummerierten Sequenzen herunter oder gib eine Fehlermeldung für entsprechende PDB-Einträge aus. getanntation erwartet als zweites Argument
    # eine Liste aus einer oder mehreren Listen. Wir benutzen pro Zeile in ab_ag jeweils eine Liste, die eine einzige Liste enthält.
    try:
        getannotation(pdb, [fab_unique], f"data/sabdab_download/{pdb}")
        # Extrahiere die CDR-Sequenzen aus den heruntergeladenen Chothia-annotierten Sequenzen und speichere sie in ab_ag
        # Speichere jede .ann-Datei in einm DataFrame
        VH_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_unique[1]}_VH.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
        VL_chothia_seq = pd.read_csv(f"data/sabdab_download/{pdb}/{pdb}_{fab_unique[2]}_VL.ann", sep='\t', header=None, names=['Chothia_Position', 'Aminosäure'])
        # Extrahiere die CDR-Sequenzen aus den Chothia-annotierten Sequenzen
        # und speichere sie in ab_ag
        H1_seq = VH_chothia_seq.loc[cdr_regions["H1"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_H1"] = H1_seq
        H2_seq = VH_chothia_seq.loc[cdr_regions["H2"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_H2"] = H2_seq
        H3_seq = VH_chothia_seq.loc[cdr_regions["H3"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_H3"] = H3_seq
        L1_seq = VL_chothia_seq.loc[cdr_regions["L1"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_L1"] = L1_seq
        L2_seq = VL_chothia_seq.loc[cdr_regions["L2"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_L2"] = L2_seq
        L3_seq = VL_chothia_seq.loc[cdr_regions["L3"], "Aminosäure"].tolist()
        ab_ag.at[x, "CDR_L3"] = L3_seq
    except Exception as e:
        print(f"Could not download Chothia-annotated sequence for {pdb}: {e}")
    shutil.rmtree(f"data/sabdab_download/{pdb}")  # Lösche den Ordner, nachdem die Sequenzen extrahiert wurden

[['9ds2', 'G', 'I', 0]]
['9ds2', 'G', 'I', 0]
[['9ds2', 'H', 'J', 0]]
['9ds2', 'H', 'J', 0]
[['9ds2', 'K', 'L', 0]]
['9ds2', 'K', 'L', 0]
[['8uzp', 'H', 'L', 0]]
['8uzp', 'H', 'L', 0]
[['8uzp', 'A', 'B', 0]]
['8uzp', 'A', 'B', 0]
[['8veb', 'G', 'I', 0]]
['8veb', 'G', 'I', 0]
[['8veb', 'H', 'L', 0]]
['8veb', 'H', 'L', 0]
[['8veb', 'J', 'K', 0]]
['8veb', 'J', 'K', 0]
[['8ved', 'H', 'L', 0]]
['8ved', 'H', 'L', 0]
[['8ved', 'G', 'I', 0]]
['8ved', 'G', 'I', 0]
[['8ved', 'J', 'K', 0]]
['8ved', 'J', 'K', 0]
[['8vee', 'H', 'L', 0]]
['8vee', 'H', 'L', 0]
[['8vee', 'G', 'I', 0]]
['8vee', 'G', 'I', 0]
[['8vee', 'J', 'K', 0]]
['8vee', 'J', 'K', 0]
[['8vef', 'H', 'L', 0]]
['8vef', 'H', 'L', 0]
[['8vef', 'J', 'K', 0]]
['8vef', 'J', 'K', 0]
[['8vef', 'G', 'I', 0]]
['8vef', 'G', 'I', 0]
[['8vpf', 'M', 'N', 0]]
['8vpf', 'M', 'N', 0]
[['8vpf', 'D', 'F', 0]]
['8vpf', 'D', 'F', 0]
[['8vpf', 'E', 'G', 0]]
['8vpf', 'E', 'G', 0]
[['9dpc', 'H', 'L', 0]]
['9dpc', 'H', 'L', 0]
[['9dru', 'H', 'L', 0]]
['9dru', '

KeyboardInterrupt: 